In [ ]:
%pip install plotly seaborn

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt

from datetime import date
from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

In [ ]:
PROJECT_NAME = "AreaVerde"

In [ ]:
def Heatmap(data, norm, colors):
  plt.figure(figsize=(20, 20))
  custom_colors = colors
  cmap = ListedColormap(custom_colors)
  sns.heatmap(
      data,
      cmap=cmap,          
      norm=norm,
      linewidths=0.6,
      fmt='d',
      cbar=False,
  )

  plt.tight_layout()
  plt.show()

# **Load Data**

In [ ]:
# Load data
project = dh.get_or_create_project(PROJECT_NAME)

spira = project.get_dataitem("spira_flow_data_2024").as_df()
spira['DateTime'] = pd.to_datetime(spira['DateTime'])

In [ ]:
spira

In [ ]:
spira_locations = spira[['spira_unique_id', 'longitudine', 'latitudine']].drop_duplicates()

# **Selected Date and Spira**

In [ ]:
filtered_df = spira[spira['spira_unique_id'].between(200, 250)]
ids = filtered_df['spira_unique_id'].unique().tolist()

In [ ]:
start = date(2024, 6, 1)
end = date(2024, 7, 30)
sel_spira = ids # This is the green section in yearly heatmap

# **One_to_Many IDs in 2023**

In [ ]:
spira_2023 = project.get_dataitem("spira_flow_data_2023").as_df()
spira_2023['DateTime'] = pd.to_datetime(spira_2023['DateTime'])

In [ ]:
spira_2023

In [ ]:
one_many_2023 = spira_2023[['spira_code', 'spira_unique_id']].groupby('spira_unique_id').agg(lambda x:  x.unique())

In [ ]:
duplicated = []
for code, ip in one_many_2023['spira_code'].items():
    if len(ip)>1:
        duplicated.append(int(code))
        print(code, ip)

In [ ]:
for code in duplicated[:3]:
    spira_2023_c = spira_2023.loc[spira_2023['spira_unique_id'] == code ].copy()
    spira_df_2023 = spira_2023_c[['spira_code','DateTime']]
    fig = px.scatter(
    spira_df_2023, 
    x='DateTime', 
    y='spira_code', 
    title=f"Spira {code} Timeline", 
    labels={'time': 'Time', 'spira': 'Spira'}, 
    color='spira_code'  
)
    fig.update_layout(
        width=700,  
        height=300,  
        title_font_size=20,
    )
    fig.show()


# **One_to_Many IDs in 2024**

In [ ]:
one_many = spira[['spira_code', 'spira_unique_id']].groupby('spira_unique_id').agg(lambda x:  x.unique())

In [ ]:
duplicated = []
for code, ip in one_many['spira_code'].items():
    if len(ip)>1:
        duplicated.append(int(code))
        print(code, ip)

In [ ]:
for code in duplicated:
    spira_2024_c = spira.loc[spira['spira_unique_id'] == code ].copy()
    spira_df_2024 = spira_2024_c[['spira_code','DateTime']]
    fig = px.scatter(
    spira_df_2024, 
    x='DateTime', 
    y='spira_code', 
    title=f"Spira {code} Timeline", 
    labels={'time': 'Time', 'spira': 'Spira'}, 
    color='spira_code'  
)
    fig.update_layout(
        width=600,  
        height=300,  
        title_font_size=20,
    )   
    fig.show()


# **Daily**

In [ ]:
spira

In [ ]:
spira['date'] = spira['DateTime'].dt.date

In [ ]:
total_vehicle_d = spira[["date", "spira_unique_id", "count"]].groupby(["date", "spira_unique_id"], as_index=False).sum()
total_vehicle_d = total_vehicle_d.pivot_table(index="date", columns="spira_unique_id", values="count", fill_value=0).astype(int)
total_vehicle_d

In [ ]:
sel_total_vehicle_d = total_vehicle_d[sel_spira]

In [ ]:
sel_total_vehicle_d

In [ ]:
norm = BoundaryNorm(boundaries=[-0.5, 0.5,max(total_vehicle_d)], ncolors=2)
custom_colors = ['red', 'green']
Heatmap(sel_total_vehicle_d, norm, custom_colors)

# **Hourly Analysis**

In [ ]:
total_vehicle_h = spira[["DateTime", "spira_unique_id", "count"]].groupby(["DateTime", "spira_unique_id"], as_index=False).sum()

total_vehicle_h = total_vehicle_h.pivot_table(index="DateTime", columns="spira_unique_id", values="count", fill_value=0).astype(int)

In [ ]:
total_vehicle_h

In [ ]:
sel_total_vehicle_h = total_vehicle_h[sel_spira]

In [ ]:
hourly_summary = (sel_total_vehicle_h > 0).groupby(sel_total_vehicle_h.index.date).sum()
hourly_status = pd.DataFrame(index=hourly_summary.index, columns=hourly_summary.columns)

hourly_status[(hourly_summary == 24)] = 'green'  # All 24 hours have non-zero data
hourly_status[(hourly_summary >= 18) & (hourly_summary < 24)] = 'yellow'  # Some hours have zero data
hourly_status[(hourly_summary > 0) & (hourly_summary < 18)] = 'orange'  # Some hours have zero data
hourly_status[(hourly_summary == 0)] = 'red'  # All hours are zero
color_mapping = {'red': 0, 'orange': 2, 'green': 1, 'yellow':3}
hourly_status = hourly_status.replace(color_mapping)
hourly_status

In [ ]:
norm = BoundaryNorm(boundaries=[-0.5, 0.5,1.5,2.5,3.5], ncolors=4)
custom_colors = ['red', 'green','orange', 'yellow']
Heatmap(hourly_status, norm, custom_colors) 

In [ ]:
total_vehicle_w = total_vehicle_h.copy().reset_index()
total_vehicle_w['Weekday'] = total_vehicle_w['DateTime'].dt.day_name()
    

In [ ]:
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
grouped_data = total_vehicle_w.groupby('Weekday').mean().reindex(weekday_order)

In [ ]:
grouped_data[sel_spira].plot(figsize=(14, 10), legend=False)

In [ ]:
start = date(2024,5,20)
end = date(2024,6,30)
sel_total_vehicle_h = total_vehicle_h[start:end][sel_spira]

In [ ]:
mean_values = sel_total_vehicle_h.mean(axis=1)

specific_spira_id = 214
specific_spira_data = sel_total_vehicle_h[specific_spira_id]

plt.figure(figsize=(16, 10))
plt.plot(sel_total_vehicle_h.index, mean_values, label='Mean of selected Spiras', color='blue')


plt.plot(sel_total_vehicle_h.index, specific_spira_data, label=f'Spira {specific_spira_id}', color='orange')
plt.legend()
plt.show()


# **Heatmap for Whole year**

In [ ]:
total_vehicle_d.T
norm = BoundaryNorm(boundaries=[-0.5, 0.5,max(total_vehicle_d)], ncolors=2)
custom_colors = ['red', 'green']
spiras_per_plot = 100
num_plots = len(total_vehicle_d.columns) // spiras_per_plot + 1
for i in range(num_plots):
    start = i * spiras_per_plot
    end = (i + 1) * spiras_per_plot
    spira_subset = total_vehicle_d.iloc[:, start:end]
    Heatmap(spira_subset.T, norm,custom_colors)